This notebook takes the .wav format CHiME dataset, pre-processes it, and then saves it to file. This removes the pre-processing bottleneck from the model training process.

In [1]:
import tensorflow as tf
import dataset
import os
import pickle

In [2]:
model_config = {
                'dataset': 'CHiME',
                'sample_rate': 16384,  # Desired sample rate of audio. Input will be resampled to this
                'n_fft': 1024,  # Number of samples in each fourier transform
                'fft_hop': 256,  # Number of samples between the start of each fourier transform
                'n_parallel_readers': 16,
                'patch_window': 256,  # Number of fourier transforms (rows) in each patch
                'patch_hop': 128,  # Number of fourier transforms between the start of each patch
                'batch_size': 1,  # Number of patches in each batch
                'n_shuffle': 1,  # Number of patches buffered before batching
                'learning_rate': 0.0001,  # The learning rate to be used by the model
                'epochs': 0,  # Number of full passes through the dataset to train for
                'normalise_mag': True,  # Are magnitude spectrograms normalised in pre-processing?
                'GPU': '0',
                'local_run': False,
                'chime_data_root': '/home/enterprise.internal.city.ac.uk/acvn728/NewCHiME/',
                'librispeech_data_root': '/data/Speech_Data/LibriSpeech/'
                }

    
train_data, val_data, test_data = dataset.prepare_datasets(model_config)

### Create directories to store the pre-processed data

In [ ]:
if not os.path.isdir('pre_processed_data'):
    os.mkdir('pre_processed_data')
for dataset in ['training','validation','test']:
    path = os.path.join('pre_processed_data', dataset)
    if not os.path.isdir(path):
        os.mkdir(path)

In [3]:
# Start session
tf_config = tf.ConfigProto()
tf_config.gpu_options.visible_device_list = str(model_config['GPU'])
sess = tf.Session(config=tf_config)

In [ ]:
train_iterator = train_data.make_one_shot_iterator()
batch = sess.run(train_iterator.get_next())

In [ ]:
dump_name

In [ ]:
counter = 0
dump_location = 'pre_processed_data'
dump_name = os.path.join(dump_location, str(counter))
pickle.dump(batch, open(dump_name, 'wb'))

### Pre-Process and pickle the data

In [7]:
dump_location = 'pre_processed_data'
#Test Data
counter = 0
test_iterator = test_data.make_one_shot_iterator()
next_item = test_iterator.get_next()

while True:
    try:
        batch = sess.run(next_item)
        dump_name = os.path.join(dump_location,'test', str(counter))
        pickle.dump(batch, open(dump_name, 'wb'))
        counter += 1
        if counter % 100 == 0:
            print('{c} batches complete'.format(c=counter))
    except tf.errors.OutOfRangeError:
        print('All done')
        break

        
        
    
    
    

/usr/local/lib/python3.5/dist-packages/tensorflow/python/data/ops/iterator_ops.py:406: UserWarning: An unusually high number of `Iterator.get_next()` calls was detected. This often indicates that `Iterator.get_next()` is being called inside a training loop, which will cause gradual slowdown and eventual resource exhaustion. If this is the case, restructure your code to call `next_element = iterator.get_next()` once outside the loop, and use `next_element` as the input to some computation that is invoked inside the loop.
  warnings.warn(GET_NEXT_CALL_WARNING_MESSAGE)


5400 batches complete
5500 batches complete
5600 batches complete
5700 batches complete
5800 batches complete
5900 batches complete
6000 batches complete
6100 batches complete
6200 batches complete
6300 batches complete
6400 batches complete
6500 batches complete
6600 batches complete
6700 batches complete
6800 batches complete
6900 batches complete
7000 batches complete
7100 batches complete
7200 batches complete
7300 batches complete
7400 batches complete
7500 batches complete
7600 batches complete
7700 batches complete
7800 batches complete
7900 batches complete
8000 batches complete
8100 batches complete
8200 batches complete
8300 batches complete
8400 batches complete
8500 batches complete
8600 batches complete
8700 batches complete
8800 batches complete
8900 batches complete
9000 batches complete
9100 batches complete
9200 batches complete
9300 batches complete
9400 batches complete
9500 batches complete
9600 batches complete
9700 batches complete
9800 batches complete
9900 batch

In [33]:
def load_pickle_file(file):
    
    mixed_spec, voice_spec, background_spec, mixed_audio, voice_audio, background_audio = pickle.load(open(file, 'rb'))
    
    
    return (mixed_spec, voice_spec, background_spec, mixed_audio, voice_audio, background_audio)

def py_load_file(path):
    
    return tf.py_func(load_pickle_file, [path], tf.float32, stateful=False)


def use_prepared_dataset(location):
    
    return (
        tf.data.Dataset.list_files(location, shuffle=False)
        .map(py_load_file)
        #.shuffle(n_shuffle).batch(batch_size).prefetch(3)
    )

In [34]:
location = 'pre_processed_data/test/*'
data = use_prepared_dataset(location)
pipe = data.make_one_shot_iterator()
next_item = pipe.get_next()
sess.run(next_item)

InvalidArgumentError: pyfunc_80 returns 6 values, but expects to see 1 values.
	 [[{{node PyFuncStateless}} = PyFuncStateless[Tin=[DT_STRING], Tout=[DT_FLOAT], token="pyfunc_80"](arg0)]]
	 [[{{node IteratorGetNext_5320}} = IteratorGetNext[output_shapes=[<unknown>], output_types=[DT_FLOAT], _device="/job:localhost/replica:0/task:0/device:CPU:0"](OneShotIterator_10)]]